# Libraries

In [3]:
%pip install -U ultralytics

^C
Note: you may need to restart the kernel to use updated packages.


In [3]:
root_dir = r"D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset"

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2

import os
import shutil
import random
import xml.etree.ElementTree as ET
import yaml
from pathlib import Path
from collections import Counter

from ultralytics import YOLO
from sklearn.model_selection import KFold

# Exploring Dataset

In [5]:
dataset_dir = r"D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET"

for root, dirs, files in os.walk(dataset_dir):
   for name in dirs:
      print(os.path.join(root, name))

D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\Annotations
D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\grayscale_images
D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\images
D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\images_resized
D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\PCB_USED
D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\rotation
D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\Annotations\Missing_hole
D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\Annotations\Mouse_bite
D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\Annotations\Open_circuit
D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\Annotations\Short
D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\Annotations\Spur
D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\Annotations\Spurious_copper
D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\images\Missing_hole
D:\BML Munjal\3rd Year\Sem 5\DIP\D

In [8]:
def count_files_in_folder(folder_path):
    # Get list of all files in the folder
    files = os.listdir(folder_path)

    # Count the number of files
    num_files = len(files)

    return num_files
subfolders = ['Missing_hole', 'Mouse_bite', 'Open_circuit', 'Short', 'Spur', 'Spurious_copper']

images_dir = os.path.join(dataset_dir, 'images')
annot_dir = os.path.join(dataset_dir, 'Annotations')

for subfolder in subfolders:
    images_path = os.path.join(images_dir, subfolder)
    annot_path = os.path.join(annot_dir, subfolder)

    print(f'{subfolder:<15} \t\
            {count_files_in_folder(images_path)} images \t\
            {count_files_in_folder(annot_path)} annotations')

Missing_hole    	            115 images 	            115 annotations
Mouse_bite      	            115 images 	            115 annotations
Open_circuit    	            116 images 	            116 annotations
Short           	            116 images 	            116 annotations
Spur            	            115 images 	            115 annotations
Spurious_copper 	            116 images 	            116 annotations


# Creating Annotations Dataframe

In [4]:
annot_dir = r"D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\Annotations"

def parse_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    data = []

    filename = root.find('filename').text
    width = int(root.find('size/width').text)
    height = int(root.find('size/height').text)

    for obj in root.findall('object'):
        name = obj.find('name').text
        xmin = int(obj.find('bndbox/xmin').text)
        ymin = int(obj.find('bndbox/ymin').text)
        xmax = int(obj.find('bndbox/xmax').text)
        ymax = int(obj.find('bndbox/ymax').text)

        data.append({
            'filename': filename,
            'width': width,
            'height': height,
            'class': name,
            'xmin': xmin,
            'ymin': ymin,
            'xmax': xmax,
            'ymax': ymax
        })

    return data
# List to store parsed data from all XML files
all_data = []

# Recursively traverse subdirectories
for root, dirs, files in os.walk(annot_dir):
    for name in files:
        if name.endswith('.xml'):
            xml_path = os.path.join(root, name)
            all_data.extend(parse_xml(xml_path))
# Create DataFrame from the parsed data
annot_df = pd.DataFrame(all_data)
annot_df.head()


,filename,width,height,class,xmin,ymin,xmax,ymax
0,01_missing_hole_01.jpg,3034,1586,missing_hole,2459,1274,2530,1329
1,01_missing_hole_01.jpg,3034,1586,missing_hole,1613,334,1679,396
2,01_missing_hole_01.jpg,3034,1586,missing_hole,1726,794,1797,854
3,01_missing_hole_02.jpg,3034,1586,missing_hole,2584,232,2650,298
4,01_missing_hole_02.jpg,3034,1586,missing_hole,2366,803,2406,860


# Resizing Images

In [11]:
dataset_dir = r"D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET"
images_dir = r"D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\images"

def resize_images(input_dir, output_dir, target_size=(640, 640)):
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Traverse through the subfolders in the input folder
    for root, _, files in os.walk(input_dir):
        for file in files:
            # Check if the file is an image
            if file.lower().endswith(('.jpg')):
                # Read the image
                image_path = os.path.join(root, file)
                image = cv2.imread(image_path)

                # Resize the image
                resized_image = cv2.resize(image, target_size)

                # Save the resized image to the output folder
                output_path = os.path.join(output_dir, file)
                cv2.imwrite(output_path, resized_image)

resized_img_dir = os.path.join(dataset_dir, 'images_resized')
resize_images(images_dir, resized_img_dir)
def resize_annotations(annot_df, target_size=(640, 640)):
    all_data = []

    # Iterate through the annotation DataFrame
    for index, row in annot_df.iterrows():

        # Resize the bounding box coordinates
        width_ratio = target_size[0] / row['width']
        height_ratio = target_size[1] / row['height']

        resized_xmin = int(row['xmin'] * width_ratio)
        resized_ymin = int(row['ymin'] * height_ratio)
        resized_xmax = int(row['xmax'] * width_ratio)
        resized_ymax = int(row['ymax'] * height_ratio)

        # Update the all data list with resized annotations
        all_data.append({
            'filename': row['filename'],
            'width': target_size[0],
            'height': target_size[1],
            'class': row['class'],
            'xmin': resized_xmin,
            'ymin': resized_ymin,
            'xmax': resized_xmax,
            'ymax': resized_ymax
        })

    annot_df_resized = pd.DataFrame(all_data)
    return annot_df_resized

annot_df_resized = resize_annotations(annot_df)
annot_df_resized.head()


,filename,width,height,class,xmin,ymin,xmax,ymax
0,01_missing_hole_01.jpg,640,640,missing_hole,518,514,533,536
1,01_missing_hole_01.jpg,640,640,missing_hole,340,134,354,159
2,01_missing_hole_01.jpg,640,640,missing_hole,364,320,379,344
3,01_missing_hole_02.jpg,640,640,missing_hole,545,93,558,120
4,01_missing_hole_02.jpg,640,640,missing_hole,499,324,507,347


# Converting RGB images to Greyscale Images (Using Histogram Equalisation)

In [12]:
# Directory containing all images categorized into types of defects
images_dir = os.path.join(dataset_dir, 'images_resized')  # Input image directory

# Directory to save grayscale images
grayscale_img_dir = os.path.join(dataset_dir, 'grayscale_images')  # Output directory for grayscale images

# Create the output directory if it doesn't exist
os.makedirs(grayscale_img_dir, exist_ok=True)
print(f"Output directory '{grayscale_img_dir}' created or already exists.")

# Function to perform histogram equalization and convert to grayscale
def process_image(image_path):
    # Read the image
    image = cv2.imread(image_path)

    # Check if the image was loaded successfully
    if image is None:
        print(f"Error loading image: {image_path}")
        return None

    # Convert to YUV color space for histogram equalization
    img_yuv = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    # Perform histogram equalization on the Y channel
    img_yuv[:, :, 0] = cv2.equalizeHist(img_yuv[:, :, 0])
    # Convert back to BGR color space
    img_eq = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)

    # Convert to grayscale
    grayscale_image = cv2.cvtColor(img_eq, cv2.COLOR_BGR2GRAY)

    return grayscale_image

# Iterate through all files in the input directory
for root, dirs, files in os.walk(images_dir):
    for filename in files:
        if filename.endswith('.jpg') or filename.endswith('.png'):  # Add other formats if necessary
            image_path = os.path.join(root, filename)
            grayscale_image = process_image(image_path)

            # Save the grayscale image
            if grayscale_image is not None:
                # Create output path preserving the directory structure
                output_image_path = os.path.join(grayscale_img_dir, os.path.relpath(image_path, images_dir))
                # Create the necessary directories in the output path
                os.makedirs(os.path.dirname(output_image_path), exist_ok=True)
                cv2.imwrite(output_image_path, grayscale_image)
                print(f"Saved grayscale image: {output_image_path}")

print("Conversion to grayscale completed.")


Output directory 'D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\grayscale_images' created or already exists.
Saved grayscale image: D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\grayscale_images\01_missing_hole_01.jpg
Saved grayscale image: D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\grayscale_images\01_missing_hole_02.jpg
Saved grayscale image: D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\grayscale_images\01_missing_hole_03.jpg
Saved grayscale image: D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\grayscale_images\01_missing_hole_04.jpg
Saved grayscale image: D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\grayscale_images\01_missing_hole_05.jpg
Saved grayscale image: D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\grayscale_images\01_missing_hole_06.jpg
Saved grayscale image: D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\grayscale_images\01_missing_hole_07.jpg
Saved grayscale image: D:\BML Munjal\3rd

# Converting Greyscale Images to Binary Images (Using Histogram Equalisation)

In [13]:
# Directory containing all images categorized into types of defects
images_dir = os.path.join(dataset_dir, 'grayscale_images')  # Input grayscale images directory

# Directory to save binary images
binary_img_dir = os.path.join(dataset_dir, 'binary_images')  # Output directory for binary images

# Create the output directory if it doesn't exist
os.makedirs(binary_img_dir, exist_ok=True)
print(f"Output directory '{binary_img_dir}' created or already exists.")

# Function to apply histogram equalization and convert to binary
def process_image(image_path, threshold=128):
    """
    Process an image by applying histogram equalization and binary thresholding.

    Args:
        image_path (str): Path to the input image.
        threshold (int): Threshold value for binary conversion.

    Returns:
        numpy.ndarray or None: Processed binary image or None if image loading fails.
    """
    # Read the grayscale image
    grayscale_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Check if the image was loaded successfully
    if grayscale_image is None:
        print(f"Error loading image: {image_path}")
        return None

    # Apply histogram equalization to enhance contrast
    equalized_image = cv2.equalizeHist(grayscale_image)

    # Apply binary thresholding
    _, binary_image = cv2.threshold(equalized_image, threshold, 255, cv2.THRESH_BINARY)

    return binary_image

# Iterate through all files in the input directory
for root, dirs, files in os.walk(images_dir):
    for filename in files:
        if filename.endswith('.jpg') or filename.endswith('.png'):  # Add other formats if necessary
            image_path = os.path.join(root, filename)
            binary_image = process_image(image_path)

            # Save the binary image
            if binary_image is not None:
                # Create output path preserving the directory structure
                output_image_path = os.path.join(binary_img_dir, os.path.relpath(image_path, images_dir))
                # Create the necessary directories in the output path
                os.makedirs(os.path.dirname(output_image_path), exist_ok=True)
                cv2.imwrite(output_image_path, binary_image)
                print(f"Saved binary image: {output_image_path}")

print("Conversion to binary completed.")


Output directory 'D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\binary_images' created or already exists.
Saved binary image: D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\binary_images\01_missing_hole_01.jpg
Saved binary image: D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\binary_images\01_missing_hole_02.jpg
Saved binary image: D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\binary_images\01_missing_hole_03.jpg
Saved binary image: D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\binary_images\01_missing_hole_04.jpg
Saved binary image: D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\binary_images\01_missing_hole_05.jpg
Saved binary image: D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\binary_images\01_missing_hole_06.jpg
Saved binary image: D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\binary_images\01_missing_hole_07.jpg
Saved binary image: D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\binary_i

# Splitting to Train, Test and Validate Datasets


Creating YOLO labels

In [ ]:
    import os
    import pandas as pd
    import xml.etree.ElementTree as ET

    # Class mapping (adjust according to your dataset)
    class_map = {
        'missing_hole': 0,
        'mouse_bite': 1,
        'open_circuit': 2,
        'short': 3,
        'spur': 4,
        'spurious_copper': 5
    }

    def parse_xml(xml_file):
        """
        Parse a single PASCAL VOC XML file and extract annotation data.
        """
        tree = ET.parse(xml_file)
        root = tree.getroot()
        data = []

        filename = root.find('filename').text
        width = int(root.find('size/width').text)
        height = int(root.find('size/height').text)

        for obj in root.findall('object'):
            name = obj.find('name').text
            xmin = int(obj.find('bndbox/xmin').text)
            ymin = int(obj.find('bndbox/ymin').text)
            xmax = int(obj.find('bndbox/xmax').text)
            ymax = int(obj.find('bndbox/ymax').text)

            data.append({
                'filename': filename,
                'width': width,
                'height': height,
                'class': name,
                'xmin': xmin,
                'ymin': ymin,
                'xmax': xmax,
                'ymax': ymax
            })

        return data

    def create_yolo_labels(data, output_dir):
        """
        Create YOLO label files from the parsed annotation data.
        """
        os.makedirs(output_dir, exist_ok=True)  # Ensure the output directory exists

        # Group data by filename to create one label file per image
        for filename, group in data.groupby('filename'):
            label_file_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}.txt")

            with open(label_file_path, 'w') as label_file:
                for _, row in group.iterrows():
                    class_id = class_map[row['class']]
                    # Convert VOC to YOLO format
                    center_x = ((row['xmin'] + row['xmax']) / 2) / row['width']
                    center_y = ((row['ymin'] + row['ymax']) / 2) / row['height']
                    bbox_width = (row['xmax'] - row['xmin']) / row['width']
                    bbox_height = (row['ymax'] - row['ymin']) / row['height']

                    # Write to label file
                    label_file.write(f"{class_id} {center_x:.6f} {center_y:.6f} {bbox_width:.6f} {bbox_height:.6f}\n")

            print(f"Created YOLO label: {label_file_path}")

    # Directory containing XML annotations
    annot_dir = r"D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\Annotations"  # Replace with your XML annotations directory path
    output_labels_dir = r"D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\yolo_labels"  # Replace with your desired output labels directory path

    # Parse all XML files and load data into a DataFrame
    all_data = []
    for root, dirs, files in os.walk(annot_dir):
        for name in files:
            if name.endswith('.xml'):
                xml_path = os.path.join(root, name)
                all_data.extend(parse_xml(xml_path))

    # Convert the list of dictionaries into a DataFrame
    annot_df = pd.DataFrame(all_data)

    # Generate YOLO labels
    create_yolo_labels(annot_df, output_labels_dir)


Created YOLO label: D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\yolo_labels\01_missing_hole_01.txt
Created YOLO label: D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\yolo_labels\01_missing_hole_02.txt
Created YOLO label: D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\yolo_labels\01_missing_hole_03.txt
Created YOLO label: D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\yolo_labels\01_missing_hole_04.txt
Created YOLO label: D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\yolo_labels\01_missing_hole_05.txt
Created YOLO label: D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\yolo_labels\01_missing_hole_06.txt
Created YOLO label: D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\yolo_labels\01_missing_hole_07.txt
Created YOLO label: D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\yolo_labels\01_missing_hole_08.txt
Created YOLO label: D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\yolo_labels\01_missing_hole

Splitting the Greyscale Images

In [21]:
import os
import random
import shutil

def split_images_and_labels(greyscale_images_dir, labels_dir, output_dir, train_split=0.8, val_split=0.1, test_split=0.1):
    """
    Split images and labels into train, validation, and test datasets.
    """

    # Ensure the splits sum to 1
    if train_split + val_split + test_split != 1:
        raise ValueError("Train, validation, and test splits must sum to 1.")

    # Create necessary directories for splits
    os.makedirs(os.path.join(output_dir, 'images/train'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'images/val'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'images/test'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'labels/train'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'labels/val'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'labels/test'), exist_ok=True)

    # Get all YOLO label files
    label_files = [f for f in os.listdir(labels_dir) if f.endswith('.txt')]

    # Shuffle the filenames for randomness
    random.shuffle(label_files)

    # Calculate split sizes
    num_files = len(label_files)
    num_train = int(num_files * train_split)
    num_val = int(num_files * val_split)
    num_test = num_files - num_train - num_val

    train_files = label_files[:num_train]
    val_files = label_files[num_train:num_train + num_val]
    test_files = label_files[num_train + num_val:]

    # Helper to save images and labels
    def save_dataset(file_list, dataset):
        for label_file in file_list:
            # Paths for the label and corresponding image
            label_path = os.path.join(labels_dir, label_file)
            image_filename = os.path.splitext(label_file)[0] + '.jpg'  # Assuming images are in .jpg format
            image_path = os.path.join(greyscale_images_dir, image_filename)

            # Copy label file
            shutil.copy(label_path, os.path.join(output_dir, f'labels/{dataset}/{label_file}'))

            # Copy corresponding image if it exists
            if os.path.exists(image_path):
                shutil.copy(image_path, os.path.join(output_dir, f'images/{dataset}/{image_filename}'))
            else:
                print(f"Warning: Image {image_filename} not found for label {label_file}.")

    # Save splits
    save_dataset(train_files, 'train')
    save_dataset(val_files, 'val')
    save_dataset(test_files, 'test')

    print("Dataset split completed!")
    print(f"Train: {len(train_files)}, Validation: {len(val_files)}, Test: {len(test_files)}")

# Directories for grayscale images and YOLO labels
greyscale_images_dir = r"D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\grayscale_images"
labels_dir = r"D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\yolo_labels"
output_dir = r"D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\split_dataset\greyscale"

# Split the dataset
split_images_and_labels(greyscale_images_dir, labels_dir, output_dir)


Dataset split completed!
Train: 554, Validation: 69, Test: 70


Splitting the Binary Images

In [22]:
import os
import random
import shutil

def split_images_and_labels(binary_images_dir, labels_dir, output_dir, train_split=0.8, val_split=0.1, test_split=0.1):
    """
    Split images and labels into train, validation, and test datasets.
    """

    # Ensure the splits sum to 1
    if train_split + val_split + test_split != 1:
        raise ValueError("Train, validation, and test splits must sum to 1.")

    # Create necessary directories for splits
    os.makedirs(os.path.join(output_dir, 'images/train'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'images/val'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'images/test'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'labels/train'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'labels/val'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'labels/test'), exist_ok=True)

    # Get all YOLO label files
    label_files = [f for f in os.listdir(labels_dir) if f.endswith('.txt')]

    # Shuffle the filenames for randomness
    random.shuffle(label_files)

    # Calculate split sizes
    num_files = len(label_files)
    num_train = int(num_files * train_split)
    num_val = int(num_files * val_split)
    num_test = num_files - num_train - num_val

    train_files = label_files[:num_train]
    val_files = label_files[num_train:num_train + num_val]
    test_files = label_files[num_train + num_val:]

    # Helper to save images and labels
    def save_dataset(file_list, dataset):
        for label_file in file_list:
            # Paths for the label and corresponding image
            label_path = os.path.join(labels_dir, label_file)
            image_filename = os.path.splitext(label_file)[0] + '.jpg'  # Assuming images are in .jpg format
            image_path = os.path.join(binary_images_dir, image_filename)

            # Copy label file
            shutil.copy(label_path, os.path.join(output_dir, f'labels/{dataset}/{label_file}'))

            # Copy corresponding image if it exists
            if os.path.exists(image_path):
                shutil.copy(image_path, os.path.join(output_dir, f'images/{dataset}/{image_filename}'))
            else:
                print(f"Warning: Image {image_filename} not found for label {label_file}.")

    # Save splits
    save_dataset(train_files, 'train')
    save_dataset(val_files, 'val')
    save_dataset(test_files, 'test')

    print("Dataset split completed!")
    print(f"Train: {len(train_files)}, Validation: {len(val_files)}, Test: {len(test_files)}")

# Directories for grayscale images and YOLO labels
binary_images_dir = r"D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\grayscale_images"
labels_dir = r"D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\yolo_labels"
output_dir = r"D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\split_dataset\binary"

# Split the dataset
split_images_and_labels(binary_images_dir, labels_dir, output_dir)


Dataset split completed!
Train: 554, Validation: 69, Test: 70


# Model Training

K-Fold Cross Validation for Greyscale Images

In [11]:
import os
import shutil
from pathlib import Path
import pandas as pd
from collections import Counter
from sklearn.model_selection import KFold
import yaml

# Define the classes (e.g., defect types in the dataset)
classes = ['missing_hole', 'mouse_bite', 'open_circuit', 'short', 'spur', 'spurious_copper']

# Map class indices
cls_idx = list(range(len(classes)))

# Dataset directory
dataset_dir = r"D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET"
output_dir = os.path.join(dataset_dir, 'split_dataset', 'greyscale')  # Update the output directory path
dataset_path = Path(output_dir)

# Step 1: Collect labels and images from the correct directories
labels = sorted(dataset_path.rglob("labels/train/*.txt"))  # All YOLO label files in 'labels/train'
images = sorted(dataset_path.rglob("images/train/*.jpg"))  # All images in 'images/train'

print(f"Number of labels found: {len(labels)}")
print(f"Number of images found: {len(images)}")

# Check if the images and labels exist
if len(labels) == 0 or len(images) == 0:
    raise ValueError("No images or labels found. Please check the paths.")

# Use base filename (without extension) as ID
indx = [l.stem for l in labels]

# Create a DataFrame for labels with class indices as columns
labels_df = pd.DataFrame([], columns=cls_idx, index=indx)

for label in labels:
    lbl_counter = Counter()

    # Read each label file
    with open(label, 'r') as lf:
        lines = lf.readlines()

    # Count class occurrences (YOLO format: class_index x_center y_center bbox_width bbox_height)
    for l in lines:
        lbl_counter[int(l.split(' ')[0])] += 1

    # Add counts to DataFrame
    labels_df.loc[label.stem] = lbl_counter

# Replace NaN values with 0.0 (indicates no instances of a class for a given image)
labels_df = labels_df.fillna(0.0)
print("Labels DataFrame:")
print(labels_df.head())

# Step 2: K-Fold Cross Validation
ksplit = 3  # Number of folds
kf = KFold(n_splits=ksplit, shuffle=True, random_state=20)  # Shuffle for randomness

# If the number of samples is 0, this will cause an error, so we check the number of labels
if len(labels_df) == 0:
    raise ValueError("No samples to split. Please check the labels.")

kfolds = list(kf.split(labels_df))
folds = [f'split_{n}' for n in range(1, ksplit + 1)]
folds_df = pd.DataFrame(index=indx, columns=folds)

# Assign train/val splits
for idx, (train, val) in enumerate(kfolds, start=1):
    folds_df[f'split_{idx}'].loc[labels_df.iloc[train].index] = 'train'
    folds_df[f'split_{idx}'].loc[labels_df.iloc[val].index] = 'val'

# Calculate label distribution for each fold
fold_lbl_distrb = pd.DataFrame(index=folds, columns=cls_idx)
for n, (train_indices, val_indices) in enumerate(kfolds, start=1):
    train_totals = labels_df.iloc[train_indices].sum()
    val_totals = labels_df.iloc[val_indices].sum()

    # To avoid division by zero, add a small value (1E-7) to the denominator
    ratio = val_totals / (train_totals + 1E-7)
    fold_lbl_distrb.loc[f'split_{n}'] = ratio

print("Label Distribution Across Folds:")
print(fold_lbl_distrb)

# Step 3: Organize Images and Labels
save_path = Path(dataset_path / f'{ksplit}fold_crossval')  # Directory to save splits
save_path.mkdir(parents=True, exist_ok=True)

# Create directories and dataset YAML files for each fold
ds_yamls = []
for split in folds_df.columns:
    split_dir = save_path / split
    split_dir.mkdir(parents=True, exist_ok=True)
    (split_dir / 'train' / 'images').mkdir(parents=True, exist_ok=True)
    (split_dir / 'train' / 'labels').mkdir(parents=True, exist_ok=True)
    (split_dir / 'val' / 'images').mkdir(parents=True, exist_ok=True)
    (split_dir / 'val' / 'labels').mkdir(parents=True, exist_ok=True)

    # Create dataset YAML file for each split
    dataset_yaml = split_dir / f'{split}_greyscale_dataset.yaml'
    ds_yamls.append(dataset_yaml)

    with open(dataset_yaml, 'w') as ds_y:
        yaml.safe_dump({
            'path': split_dir.as_posix(),
            'train': 'train',
            'val': 'val',
            'names': classes
        }, ds_y)

# Step 4: Copy Images and Labels to Folds
for image, label in zip(images, labels):
    for split, k_split in folds_df.loc[image.stem].items():
        # Destination directories
        img_to_path = save_path / split / k_split / 'images'
        lbl_to_path = save_path / split / k_split / 'labels'

        # Copy image and label files to their respective directories
        shutil.copy(image, img_to_path / image.name)
        shutil.copy(label, lbl_to_path / label.name)

# Step 5: Save Summary CSV Files
folds_df.to_csv(save_path / "kfold_datasplit.csv")
fold_lbl_distrb.to_csv(save_path / "kfold_label_distribution.csv")

print("K-Fold Cross Validation setup completed successfully.")


Number of labels found: 554
Number of images found: 554
Labels DataFrame:
                      0    1    2    3    4    5
01_missing_hole_01  3.0  0.0  0.0  0.0  0.0  0.0
01_missing_hole_03  3.0  0.0  0.0  0.0  0.0  0.0
01_missing_hole_04  3.0  0.0  0.0  0.0  0.0  0.0
01_missing_hole_05  4.0  0.0  0.0  0.0  0.0  0.0
01_missing_hole_06  3.0  0.0  0.0  0.0  0.0  0.0
Label Distribution Across Folds:
                0         1         2         3         4         5
split_1  0.464684  0.456274  0.642553  0.416058  0.437037  0.592453
split_2  0.539062  0.467433  0.462121   0.48659  0.651064  0.455172
split_3  0.498099  0.582645  0.413919  0.609959  0.431734  0.460208
K-Fold Cross Validation setup completed successfully.


K-Fold Cross Validation for Binary Images

In [3]:
import os
import shutil
from pathlib import Path
import pandas as pd
from collections import Counter
from sklearn.model_selection import KFold
import yaml

# Define the classes (e.g., defect types in the dataset)
classes = ['missing_hole', 'mouse_bite', 'open_circuit', 'short', 'spur', 'spurious_copper']

# Map class indices
cls_idx = list(range(len(classes)))

# Dataset directory
dataset_dir = r"D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET"
output_dir = os.path.join(dataset_dir, 'split_dataset', 'binary')  # Updated the path to binary folder
dataset_path = Path(output_dir)

# Step 1: Collect labels and images from the correct directories
labels = sorted(dataset_path.rglob("labels/train/*.txt"))  # All YOLO label files in 'labels/train'
images = sorted(dataset_path.rglob("images/train/*.jpg"))  # All images in 'images/train'

print(f"Number of labels found: {len(labels)}")
print(f"Number of images found: {len(images)}")

# Check if the images and labels exist
if len(labels) == 0 or len(images) == 0:
    raise ValueError("No images or labels found. Please check the paths.")

# Use base filename (without extension) as ID
indx = [l.stem for l in labels]

# Create a DataFrame for labels with class indices as columns
labels_df = pd.DataFrame([], columns=cls_idx, index=indx)

for label in labels:
    lbl_counter = Counter()

    # Read each label file
    with open(label, 'r') as lf:
        lines = lf.readlines()

    # Count class occurrences (YOLO format: class_index x_center y_center bbox_width bbox_height)
    for l in lines:
        lbl_counter[int(l.split(' ')[0])] += 1

    # Add counts to DataFrame
    labels_df.loc[label.stem] = lbl_counter

# Replace NaN values with 0.0 (indicates no instances of a class for a given image)
labels_df = labels_df.fillna(0.0)
print("Labels DataFrame:")
print(labels_df.head())

# Step 2: K-Fold Cross Validation
ksplit = 3  # Number of folds
kf = KFold(n_splits=ksplit, shuffle=True, random_state=20)  # Shuffle for randomness

# If the number of samples is 0, this will cause an error, so we check the number of labels
if len(labels_df) == 0:
    raise ValueError("No samples to split. Please check the labels.")

kfolds = list(kf.split(labels_df))
folds = [f'split_{n}' for n in range(1, ksplit + 1)]
folds_df = pd.DataFrame(index=indx, columns=folds)

# Assign train/val splits
for idx, (train, val) in enumerate(kfolds, start=1):
    folds_df[f'split_{idx}'].loc[labels_df.iloc[train].index] = 'train'
    folds_df[f'split_{idx}'].loc[labels_df.iloc[val].index] = 'val'

# Calculate label distribution for each fold
fold_lbl_distrb = pd.DataFrame(index=folds, columns=cls_idx)
for n, (train_indices, val_indices) in enumerate(kfolds, start=1):
    train_totals = labels_df.iloc[train_indices].sum()
    val_totals = labels_df.iloc[val_indices].sum()

    # To avoid division by zero, add a small value (1E-7) to the denominator
    ratio = val_totals / (train_totals + 1E-7)
    fold_lbl_distrb.loc[f'split_{n}'] = ratio

print("Label Distribution Across Folds:")
print(fold_lbl_distrb)

# Step 3: Organize Images and Labels
save_path = Path(dataset_path / f'{ksplit}fold_crossval')  # Directory to save splits
save_path.mkdir(parents=True, exist_ok=True)

# Create directories and dataset YAML files for each fold
ds_yamls = []
for split in folds_df.columns:
    split_dir = save_path / split
    split_dir.mkdir(parents=True, exist_ok=True)
    (split_dir / 'train' / 'images').mkdir(parents=True, exist_ok=True)
    (split_dir / 'train' / 'labels').mkdir(parents=True, exist_ok=True)
    (split_dir / 'val' / 'images').mkdir(parents=True, exist_ok=True)
    (split_dir / 'val' / 'labels').mkdir(parents=True, exist_ok=True)

    # Create dataset YAML file for each split
    dataset_yaml = split_dir / f'{split}_binary_dataset.yaml'
    ds_yamls.append(dataset_yaml)

    with open(dataset_yaml, 'w') as ds_y:
        yaml.safe_dump({
            'path': split_dir.as_posix(),
            'train': 'train',
            'val': 'val',
            'names': classes
        }, ds_y)

# Step 4: Copy Images and Labels to Folds
for image, label in zip(images, labels):
    for split, k_split in folds_df.loc[image.stem].items():
        # Destination directories
        img_to_path = save_path / split / k_split / 'images'
        lbl_to_path = save_path / split / k_split / 'labels'

        # Copy image and label files to their respective directories
        shutil.copy(image, img_to_path / image.name)
        shutil.copy(label, lbl_to_path / label.name)

# Step 5: Save Summary CSV Files
folds_df.to_csv(save_path / "kfold_datasplit.csv")
fold_lbl_distrb.to_csv(save_path / "kfold_label_distribution.csv")

print("K-Fold Cross Validation setup completed successfully.")


Number of labels found: 554
Number of images found: 554


C:\Users\prana\AppData\Local\Temp\ipykernel_23932\3897760524.py:52: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  labels_df = labels_df.fillna(0.0)
C:\Users\prana\AppData\Local\Temp\ipykernel_23932\3897760524.py:70: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` inst

Labels DataFrame:
                      0    1    2    3    4    5
01_missing_hole_01  3.0  0.0  0.0  0.0  0.0  0.0
01_missing_hole_03  3.0  0.0  0.0  0.0  0.0  0.0
01_missing_hole_04  3.0  0.0  0.0  0.0  0.0  0.0
01_missing_hole_05  4.0  0.0  0.0  0.0  0.0  0.0
01_missing_hole_08  3.0  0.0  0.0  0.0  0.0  0.0
Label Distribution Across Folds:
                0         1         2         3         4         5
split_1  0.528958  0.386986  0.555102  0.460937  0.640167   0.47079
split_2  0.546875  0.613546  0.488281  0.449612  0.405018  0.523132
split_3  0.429603  0.516854   0.45977  0.598291  0.473684  0.507042
K-Fold Cross Validation setup completed successfully.


YOLO on Greyscale Images


In [5]:
from ultralytics import YOLO

model = YOLO('yolov8s.pt')
results = {}

# Additional arguments
batch = 16
project = 'pcb'
epochs = 10
imgsz = 640
save_period = 1
verbose = True
mixup = 0.3

# Path to the parent folder containing all splits
parent_folder_path = "D:/BML Munjal/3rd Year/Sem 5/DIP/DIP-dataset/PCB_DATASET/split_dataset/greyscale/3fold_crossval"

# Loop for each split (3 splits in this case)
for k in range(ksplit):
    # Get the path to the current split's dataset YAML file
    dataset_yaml = os.path.join(parent_folder_path, f"split_{k+1}", f"split_{k+1}_greyscale_dataset.yaml")
    
    # Ensure the YAML file exists before training
    if not os.path.exists(dataset_yaml):
        raise FileNotFoundError(f"Dataset YAML file not found: {dataset_yaml}")
    
    # Train the model on the current split
    model.train(data=dataset_yaml,  # Pointing directly to the dataset.yaml for the current split
                epochs=epochs,
                batch=batch,
                lr0=0.001,
                lrf=0.0001,
                imgsz=imgsz,
                save_period=save_period,
                verbose=verbose,
                project=project,
                mixup=mixup)
    
    # Save the results for the current fold
    results[k] = model.metrics

print("Training completed for all folds.")


New https://pypi.org/project/ultralytics/8.3.36 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.35  Python-3.12.4 torch-2.5.1+cpu CPU (11th Gen Intel Core(TM) i5-11400H 2.70GHz)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=D:/BML Munjal/3rd Year/Sem 5/DIP/DIP-dataset/PCB_DATASET/split_dataset/greyscale/3fold_crossval\split_1\split_1_greyscale_dataset.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=1, cache=False, device=None, workers=8, project=pcb, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, 

train: Scanning D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\split_dataset\greyscale\3fold_crossval\split_1\train\labels.cache... 369 images, 0 backgrounds, 0 corrupt: 100%|██████████| 369/369 [00:00<?, ?it/s]
val: Scanning D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\split_dataset\greyscale\3fold_crossval\split_1\val\labels.cache... 185 images, 0 backgrounds, 0 corrupt: 100%|██████████| 185/185 [00:00<?, ?it/s]

Plotting labels to pcb\train8\labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.001' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to pcb\train8
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10         0G      3.826      16.58      1.948          5        640: 100%|██████████| 24/24 [03:00<00:00,  7.52s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/6 [00:00<?, ?it/s]

...

      10/10         0G      1.529     0.8665     0.9367          7        640: 100%|██████████| 24/24 [02:58<00:00,  7.45s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:34<00:00,  5.70s/it]

                   all        184        782       0.94      0.874      0.922      0.481



10 epochs completed in 0.593 hours.
Optimizer stripped from pcb\train822\weights\last.pt, 22.5MB
Optimizer stripped from pcb\train822\weights\best.pt, 22.5MB

Validating pcb\train822\weights\best.pt...
Ultralytics 8.3.35  Python-3.12.4 torch-2.5.1+cpu CPU (11th Gen Intel Core(TM) i5-11400H 2.70GHz)
Model summary (fused): 168 layers, 11,127,906 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:31<00:00,  5.18s/it]


                   all        184        782      0.939      0.871      0.921      0.481
          missing_hole         29        131      0.963      0.977      0.977      0.594
            mouse_bite         31        141      0.941      0.887      0.944      0.472
          open_circuit         29        113      0.956       0.85      0.893      0.427
                 short         36        147      0.957      0.915      0.972      0.494
                  spur         29        117      0.858      0.692       0.78      0.392
       spurious_copper         30        133       0.96      0.908      0.962      0.508
Speed: 3.1ms preprocess, 156.3ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to pcb\train822
Training completed for all folds.


YOLO on Binary Images

In [6]:
from ultralytics import YOLO

model = YOLO('yolov8s.pt')
results = {}

# Additional arguments
batch = 16
project = 'pcb'
epochs = 10
imgsz = 640
save_period = 1
verbose = True
mixup = 0.3

# Path to the parent folder containing all splits
parent_folder_path = "D:/BML Munjal/3rd Year/Sem 5/DIP/DIP-dataset/PCB_DATASET/split_dataset/binary/3fold_crossval"

# Loop for each split (3 splits in this case)
for k in range(ksplit):
    # Get the path to the current split's dataset YAML file
    dataset_yaml = os.path.join(parent_folder_path, f"split_{k+1}", f"split_{k+1}_binary_dataset.yaml")
    
    # Ensure the YAML file exists before training
    if not os.path.exists(dataset_yaml):
        raise FileNotFoundError(f"Dataset YAML file not found: {dataset_yaml}")
    
    # Train the model on the current split
    model.train(data=dataset_yaml,  # Pointing directly to the dataset.yaml for the current split
                epochs=epochs,
                batch=batch,
                lr0=0.001,
                lrf=0.0001,
                imgsz=imgsz,
                save_period=save_period,
                verbose=verbose,
                project=project,
                mixup=mixup)
    
    # Save the results for the current fold
    results[k] = model.metrics

print("Training completed for all folds.")


New https://pypi.org/project/ultralytics/8.3.36 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.35  Python-3.12.4 torch-2.5.1+cpu CPU (11th Gen Intel Core(TM) i5-11400H 2.70GHz)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=D:/BML Munjal/3rd Year/Sem 5/DIP/DIP-dataset/PCB_DATASET/split_dataset/binary/3fold_crossval\split_1\split_1_binary_dataset.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=1, cache=False, device=None, workers=8, project=pcb, name=train9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visual

train: Scanning D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\split_dataset\binary\3fold_crossval\split_1\train\labels... 369 images, 0 backgrounds, 0 corrupt: 100%|██████████| 369/369 [00:00<00:00, 1965.59it/s]

train: New cache created: D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\split_dataset\binary\3fold_crossval\split_1\train\labels.cache



val: Scanning D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\split_dataset\binary\3fold_crossval\split_1\val\labels... 185 images, 0 backgrounds, 0 corrupt: 100%|██████████| 185/185 [00:00<00:00, 2007.80it/s]

val: New cache created: D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\split_dataset\binary\3fold_crossval\split_1\val\labels.cache
Plotting labels to pcb\train9\labels.jpg... 


...

      10/10         0G      1.541     0.8314     0.9449          8        640: 100%|██████████| 24/24 [02:57<00:00,  7.42s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:33<00:00,  5.64s/it]

                   all        184        787      0.945      0.863      0.914       0.48



10 epochs completed in 0.597 hours.
Optimizer stripped from pcb\train922\weights\last.pt, 22.5MB
Optimizer stripped from pcb\train922\weights\best.pt, 22.5MB

Validating pcb\train922\weights\best.pt...
Ultralytics 8.3.35  Python-3.12.4 torch-2.5.1+cpu CPU (11th Gen Intel Core(TM) i5-11400H 2.70GHz)
Model summary (fused): 168 layers, 11,127,906 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:30<00:00,  5.10s/it]


                   all        184        787      0.936      0.859      0.908      0.488
          missing_hole         27        119      0.983      0.995       0.99      0.615
            mouse_bite         30        138      0.889      0.891      0.914      0.455
          open_circuit         30        120      0.961      0.811      0.884      0.462
                 short         33        140      0.948      0.907      0.951      0.495
                  spur         31        126      0.898        0.7       0.78      0.392
       spurious_copper         33        144      0.939       0.85      0.929      0.509
Speed: 2.9ms preprocess, 154.1ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to pcb\train922
Training completed for all folds.


Model Training for greyscale


In [ ]:
model = YOLO('yolov8s.pt')

# Configuration Parameters
batch = 16
project = 'pcb_project_greyscale'
epochs = 20
imgsz = 640
save_period = 1
verbose = True
mixup = 0.3  # Image blending augmentation

# Define paths for dataset YAML
output_dir = r"D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\split_dataset\greyscale" # Replace with the path to your dataset directory
root_dir = r"D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset"  # Replace with the path to your project directory

# Prepare YAML content
all_data_yaml = f"""
path: {output_dir}
train: images/train
val: images/val

names:
    0: missing_hole
    1: mouse_bite
    2: open_circuit
    3: short
    4: spur 
    5: spurious_copper
"""

# Save YAML file
data_path = os.path.join(root_dir, 'greyscale_dataset.yaml')  # Updated filename to avoid confusion
with open(data_path, 'w') as f:
    f.write(all_data_yaml)

# Train the model using the defined YAML
result = model.train(data=data_path,
                     epochs=epochs,
                     batch=batch,
                     lr0=0.001,
                     lrf=0.0001,
                     imgsz=imgsz,
                     save_period=save_period,
                     verbose=verbose,
                     project=project,
                     mixup=mixup)


Ultralytics 8.3.36  Python-3.12.4 torch-2.5.1+cpu CPU (11th Gen Intel Core(TM) i5-11400H 2.70GHz)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\greyscale_dataset.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=1, cache=False, device=None, workers=8, project=pcb_project_greyscale, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_

train: Scanning D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\split_dataset\greyscale\labels\train.cache... 554 images, 0 backgrounds, 0 corrupt: 100%|██████████| 554/554 [00:00<?, ?it/s]
val: Scanning D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\split_dataset\greyscale\labels\val.cache... 69 images, 0 backgrounds, 0 corrupt: 100%|██████████| 69/69 [00:00<?, ?it/s]

Plotting labels to pcb_project_greyscale\train\labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.001' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to pcb_project_greyscale\train
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/20         0G      3.841      12.13      1.815         75        640: 100%|██████████| 35/35 [04:42<00:00,  8.06s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:20<00:00,  6.86s/it]

...

      20/20         0G      1.544     0.8338     0.9382         45        640: 100%|██████████| 35/35 [04:27<00:00,  7.64s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:12<00:00,  4.23s/it]

                   all         69        286      0.965      0.823      0.919      0.481



20 epochs completed in 1.593 hours.
Optimizer stripped from pcb_project_greyscale\train\weights\last.pt, 22.5MB
Optimizer stripped from pcb_project_greyscale\train\weights\best.pt, 22.5MB

Validating pcb_project_greyscale\train\weights\best.pt...
Ultralytics 8.3.36  Python-3.12.4 torch-2.5.1+cpu CPU (11th Gen Intel Core(TM) i5-11400H 2.70GHz)
Model summary (fused): 168 layers, 11,127,906 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:10<00:00,  3.67s/it]


                   all         69        286      0.965      0.823      0.919      0.481
          missing_hole         10         47      0.988      0.979      0.995      0.629
            mouse_bite         21         81       0.93      0.655      0.841      0.401
          open_circuit         10         42       0.98      0.738      0.864      0.428
                 short         11         46      0.952      0.826      0.933      0.499
                  spur          8         29      0.941      0.862       0.92      0.447
       spurious_copper          9         41          1      0.877      0.964      0.483
Speed: 2.0ms preprocess, 148.2ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to pcb_project_greyscale\train


Model Training for Binary Images


In [5]:
model = YOLO('yolov8s.pt')

# Configuration Parameters
batch = 16
project = 'pcb_project'
epochs = 100
imgsz = 640
save_period = 1
verbose = True
mixup = 0.3  # Image blending augmentation

# Define paths for dataset YAML
output_dir = r"D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\split_dataset\binary" # Replace with the path to your dataset directory
root_dir = r"D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset"  # Replace with the path to your project directory

# Prepare YAML content
all_data_yaml = f"""
path: {output_dir}
train: images/train
val: images/val

names:
    0: missing_hole
    1: mouse_bite
    2: open_circuit
    3: short
    4: spur
    5: spurious_copper
"""

# Save YAML file
data_path = os.path.join(root_dir, 'binary_dataset.yaml')  # Updated filename to avoid confusion
with open(data_path, 'w') as f:
    f.write(all_data_yaml)

# Train the model using the defined YAML
result = model.train(data=data_path,
                     epochs=epochs,
                     batch=batch,
                     lr0=0.001,
                     lrf=0.0001,
                     imgsz=imgsz,
                     save_period=save_period,
                     verbose=verbose,
                     project=project,
                     mixup=mixup)


Ultralytics 8.3.36  Python-3.12.4 torch-2.5.1+cpu CPU (11th Gen Intel Core(TM) i5-11400H 2.70GHz)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\binary_dataset.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=1, cache=False, device=None, workers=8, project=pcb_project, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False,

train: Scanning D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\split_dataset\binary\labels\train.cache... 554 images, 0 backgrounds, 0 corrupt: 100%|██████████| 554/554 [00:00<?, ?it/s]
val: Scanning D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\split_dataset\binary\labels\val.cache... 69 images, 0 backgrounds, 0 corrupt: 100%|██████████| 69/69 [00:00<?, ?it/s]


Plotting labels to pcb_project\train2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.001' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to pcb_project\train2
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100         0G      3.747      10.61       1.84         70        640: 100%|██████████| 35/35 [04:35<00:00,  7.86s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:15<00:00,  5.09s/it]

                   all         69        283      0.678     0.0189    0.00438    0.00174


...

    100/100         0G      1.212     0.5752     0.8717         40        640: 100%|██████████| 35/35 [04:31<00:00,  7.77s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:13<00:00,  4.41s/it]

                   all         69        283       0.95      0.944      0.965      0.555



100 epochs completed in 7.911 hours.
Optimizer stripped from pcb_project\train2\weights\last.pt, 22.5MB
Optimizer stripped from pcb_project\train2\weights\best.pt, 22.5MB

Validating pcb_project\train2\weights\best.pt...
Ultralytics 8.3.36  Python-3.12.4 torch-2.5.1+cpu CPU (11th Gen Intel Core(TM) i5-11400H 2.70GHz)
Model summary (fused): 168 layers, 11,127,906 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:10<00:00,  3.63s/it]


                   all         69        283      0.955      0.926      0.963      0.564
          missing_hole         11         44          1      0.962      0.995      0.649
            mouse_bite         11         43          1      0.906      0.979      0.582
          open_circuit         11         43      0.958      0.977      0.989      0.543
                 short         12         51          1      0.957      0.994      0.548
                  spur         17         71      0.949      0.789      0.888      0.482
       spurious_copper          7         31      0.824      0.968      0.931      0.579
Speed: 1.6ms preprocess, 146.7ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to pcb_project\train2


# Testing

Testing for greyscale images

In [19]:
import os
import shutil
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from ultralytics import YOLO

# Load the trained model
best_model_path = r"D:\BML Munjal\3rd Year\Sem 5\DIP\pcb_project_greyscale\train\weights\best.pt"  # Update fold if needed
model = YOLO(best_model_path)

# Define the directory containing the test dataset
test_data_dir = r"D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\split_dataset\greyscale\images\test"  # Adjust split as required

# Run inference using the model
metrics = model(source=test_data_dir, 
                imgsz=640, 
                conf=0.25, 
                save=True, 
                save_txt=True, 
                save_conf=True)

# Paths for prediction files and results directory
predict_dir = r"D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\Final_Results"
dest_predict_dir = os.path.join(predict_dir, 'greyscale')

# Copy the prediction results to a designated directory
if os.path.exists(dest_predict_dir):
    shutil.rmtree(dest_predict_dir)
shutil.copytree(predict_dir, dest_predict_dir)

# Define the class names (you can adjust the class names based on your dataset)
classes = ['missing_hole', 'mouse_bite', 'open_circuit', 'short', 'spur', 'spurious_copper']

# Assuming annot_df is your DataFrame with the original annotation information
# You would have a CSV or another structure with annotations that include filenames, width, and height
# Example: annot_df = pd.read_csv('annotations.csv')  # Load the CSV containing annotations

# Function to convert YOLO labels to original annotation format
def yolo_to_original_annot(image_name, yolo_labels, annot_df, classes):
    original_annot = []

    for yolo_label in yolo_labels:
        # Extract original width and height from annotation DataFrame
        original_size = annot_df.loc[annot_df['filename'] == image_name, ['width', 'height']].iloc[0]
        original_width, original_height = original_size['width'], original_size['height']

        # Check if YOLO label contains 5 values (without confidence)
        if len(yolo_label) == 5:
            class_index, x_center, y_center, bbox_width, bbox_height = yolo_label
            confidence = 1.0  # If confidence is missing, assume it's 1.0
        else:
            # Handle the case where there are 6 values (including confidence)
            class_index, x_center, y_center, bbox_width, bbox_height, confidence = yolo_label

        # Scale bounding box coordinates and dimensions to original size
        original_x_center = x_center * original_width
        original_y_center = y_center * original_height
        original_bbox_width = bbox_width * original_width
        original_bbox_height = bbox_height * original_height

        # Calculate original bounding box coordinates
        original_x_min = original_x_center - original_bbox_width / 2
        original_y_min = original_y_center - original_bbox_height / 2
        original_x_max = original_x_center + original_bbox_width / 2
        original_y_max = original_y_center + original_bbox_height / 2

        # Append original annotation to list
        original_annot.append({
            'filename': image_name,
            'width': int(original_width),
            'height': int(original_height),
            'class': classes[int(class_index)],
            'xmin': int(original_x_min),
            'ymin': int(original_y_min),
            'xmax': int(original_x_max),
            'ymax': int(original_y_max),
            'confidence': confidence
        })

    return pd.DataFrame(original_annot)

# Function to read YOLO format labels from file
def read_yolo_labels_from_file(file_path):
    labels = []
    with open(file_path, 'r') as file:
        for line in file:
            values = line.strip().split()
            values = [float(value) for value in values]
            labels.append(values)
    return labels

def visualize_annotations(image_name, images_dir, annot_df, pred_annot_df, is_subfolder=True):
    """
    Visualizes annotations (ground truth and predictions) for a given image.

    Parameters:
    - image_name: Name of the image file (e.g., '01_missing_hole_10.jpg').
    - images_dir: Directory containing the images.
    - annot_df: DataFrame with ground truth annotations.
    - pred_annot_df: DataFrame with YOLO predictions.
    - is_subfolder: Whether the image is stored in a subfolder.
    """
    # Construct the full image path
    if is_subfolder:
        image_path = os.path.join(images_dir, image_name)
    else:
        image_path = image_name

    # Debugging: Print the image path
    print(f"Image path: {image_path}")
    
    # Check if the image file exists
    if not os.path.exists(image_path):
        print(f"Error: Image '{image_path}' does not exist.")
        return
    
    # Load the image using OpenCV
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Failed to load image '{image_path}'.")
        return

    # Convert the image from BGR (OpenCV default) to RGB (matplotlib compatible)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Plot the image
    fig, ax = plt.subplots(1, figsize=(12, 12))
    ax.imshow(image_rgb)
    ax.axis("off")  # Hide axis for a cleaner display
    
    # Annotate ground truth boxes
    if not annot_df.empty:
        gt_annotations = annot_df[annot_df['filename'] == image_name]
        for _, row in gt_annotations.iterrows():
            class_name = row['class']
            xmin, ymin, xmax, ymax = row['xmin'], row['ymin'], row['xmax'], row['ymax']
            # Add a green rectangle for ground truth
            rect = patches.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, 
                                      linewidth=2, edgecolor='green', facecolor='none')
            ax.add_patch(rect)
            ax.text(xmin, ymin - 5, f"{class_name}", color='green', fontsize=10, 
                    bbox=dict(facecolor='white', alpha=0.7))
    
    # Annotate prediction boxes
    if not pred_annot_df.empty:
        pred_annotations = pred_annot_df[pred_annot_df['filename'] == image_name]
        for _, row in pred_annotations.iterrows():
            class_name = row['class']
            xmin, ymin, xmax, ymax = row['xmin'], row['ymin'], row['xmax'], row['ymax']
            confidence = row['confidence']
            # Add a red rectangle for predictions
            rect = patches.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, 
                                      linewidth=2, edgecolor='red', facecolor='none')
            ax.add_patch(rect)
            ax.text(xmin, ymin - 5, f"{class_name} ({confidence:.2f})", color='red', fontsize=10, 
                    bbox=dict(facecolor='white', alpha=0.7))
    
    # Display the image with annotations
    plt.show()


# Process a specific prediction file
file_path = r"D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\yolo_labels\01_missing_hole_10.txt"  # Update file path as needed
yolo_labels = read_yolo_labels_from_file(file_path)

# Assuming annot_df is your DataFrame with annotations (e.g., filename, width, height columns)
# annot_df = pd.read_csv('annotations.csv')  # Load the CSV containing annotations with filenames

# Convert predictions to the original annotation format
pred_annot_df = yolo_to_original_annot('01_missing_hole_10.jpg', yolo_labels, annot_df, classes)

# Display the first few rows of the converted DataFrame
print(pred_annot_df.head())

# Visualization functions
visualize_annotations('01_missing_hole_10.jpg', test_data_dir, annot_df, pred_annot_df, is_subfolder=True)

print("Inference, conversion, and visualization completed.")



image 1/70 D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\split_dataset\greyscale\images\test\01_missing_hole_10.jpg: 640x640 3 missing_holes, 133.5ms
image 2/70 D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\split_dataset\greyscale\images\test\01_missing_hole_19.jpg: 640x640 3 missing_holes, 118.7ms
image 3/70 D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\split_dataset\greyscale\images\test\01_mouse_bite_15.jpg: 640x640 2 mouse_bites, 1 open_circuit, 119.0ms
image 4/70 D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\split_dataset\greyscale\images\test\01_open_circuit_01.jpg: 640x640 2 open_circuits, 124.6ms
image 5/70 D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\split_dataset\greyscale\images\test\01_open_circuit_14.jpg: 640x640 2 open_circuits, 114.9ms
image 6/70 D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\split_dataset\greyscale\images\test\01_open_circuit_18.jpg: 640x640 2 open_circuits, 123.7ms
image 7/70 D:\BML Mun

<Figure size 1200x1200 with 1 Axes>

Inference, conversion, and visualization completed.


In [ ]:
import os
import shutil
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from ultralytics import YOLO

# Load the trained model
best_model_path = r"D:\BML Munjal\3rd Year\Sem 5\DIP\pcb_project_binary\train\weights\best.pt"  # Update fold if needed
model = YOLO(best_model_path)

# Define the directory containing the test dataset
test_data_dir = r"D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\split_dataset\binary\images\test"  # Adjust split as required

# Run inference using the model
metrics = model(source=test_data_dir, 
                imgsz=640, 
                conf=0.25, 
                save=True, 
                save_txt=True, 
                save_conf=True)

# Paths for prediction files and results directory
predict_dir = r"D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\Final_Results"
dest_predict_dir = os.path.join(predict_dir, 'binary')

# Copy the prediction results to a designated directory
if os.path.exists(dest_predict_dir):
    shutil.rmtree(dest_predict_dir)
shutil.copytree(predict_dir, dest_predict_dir)

# Define the class names (you can adjust the class names based on your dataset)
classes = ['missing_hole', 'mouse_bite', 'open_circuit', 'short', 'spur', 'spurious_copper']

# Assuming annot_df is your DataFrame with the original annotation information
# You would have a CSV or another structure with annotations that include filenames, width, and height
# Example: annot_df = pd.read_csv('annotations.csv')  # Load the CSV containing annotations

# Function to convert YOLO labels to original annotation format
def yolo_to_original_annot(image_name, yolo_labels, annot_df, classes):
    original_annot = []

    for yolo_label in yolo_labels:
        # Extract original width and height from annotation DataFrame
        original_size = annot_df.loc[annot_df['filename'] == image_name, ['width', 'height']].iloc[0]
        original_width, original_height = original_size['width'], original_size['height']

        # Check if YOLO label contains 5 values (without confidence)
        if len(yolo_label) == 5:
            class_index, x_center, y_center, bbox_width, bbox_height = yolo_label
            confidence = 1.0  # If confidence is missing, assume it's 1.0
        else:
            # Handle the case where there are 6 values (including confidence)
            class_index, x_center, y_center, bbox_width, bbox_height, confidence = yolo_label

        # Scale bounding box coordinates and dimensions to original size
        original_x_center = x_center * original_width
        original_y_center = y_center * original_height
        original_bbox_width = bbox_width * original_width
        original_bbox_height = bbox_height * original_height

        # Calculate original bounding box coordinates
        original_x_min = original_x_center - original_bbox_width / 2
        original_y_min = original_y_center - original_bbox_height / 2
        original_x_max = original_x_center + original_bbox_width / 2
        original_y_max = original_y_center + original_bbox_height / 2

        # Append original annotation to list
        original_annot.append({
            'filename': image_name,
            'width': int(original_width),
            'height': int(original_height),
            'class': classes[int(class_index)],
            'xmin': int(original_x_min),
            'ymin': int(original_y_min),
            'xmax': int(original_x_max),
            'ymax': int(original_y_max),
            'confidence': confidence
        })

    return pd.DataFrame(original_annot)

# Function to read YOLO format labels from file
def read_yolo_labels_from_file(file_path):
    labels = []
    with open(file_path, 'r') as file:
        for line in file:
            values = line.strip().split()
            values = [float(value) for value in values]
            labels.append(values)
    return labels

def visualize_annotations(image_name, images_dir, annot_df, pred_annot_df, is_subfolder=True):
    """
    Visualizes annotations (ground truth and predictions) for a given image.

    Parameters:
    - image_name: Name of the image file (e.g., '01_missing_hole_10.jpg').
    - images_dir: Directory containing the images.
    - annot_df: DataFrame with ground truth annotations.
    - pred_annot_df: DataFrame with YOLO predictions.
    - is_subfolder: Whether the image is stored in a subfolder.
    """
    # Construct the full image path
    if is_subfolder:
        image_path = os.path.join(images_dir, image_name)
    else:
        image_path = image_name

    # Debugging: Print the image path
    print(f"Image path: {image_path}")
    
    # Check if the image file exists
    if not os.path.exists(image_path):
        print(f"Error: Image '{image_path}' does not exist.")
        return
    
    # Load the image using OpenCV
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Failed to load image '{image_path}'.")
        return

    # Convert the image from BGR (OpenCV default) to RGB (matplotlib compatible)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Plot the image
    fig, ax = plt.subplots(1, figsize=(12, 12))
    ax.imshow(image_rgb)
    ax.axis("off")  # Hide axis for a cleaner display
    
    # Annotate ground truth boxes
    if not annot_df.empty:
        gt_annotations = annot_df[annot_df['filename'] == image_name]
        for _, row in gt_annotations.iterrows():
            class_name = row['class']
            xmin, ymin, xmax, ymax = row['xmin'], row['ymin'], row['xmax'], row['ymax']
            # Add a green rectangle for ground truth
            rect = patches.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, 
                                      linewidth=2, edgecolor='green', facecolor='none')
            ax.add_patch(rect)
            ax.text(xmin, ymin - 5, f"{class_name}", color='green', fontsize=10, 
                    bbox=dict(facecolor='white', alpha=0.7))
    
    # Annotate prediction boxes
    if not pred_annot_df.empty:
        pred_annotations = pred_annot_df[pred_annot_df['filename'] == image_name]
        for _, row in pred_annotations.iterrows():
            class_name = row['class']
            xmin, ymin, xmax, ymax = row['xmin'], row['ymin'], row['xmax'], row['ymax']
            confidence = row['confidence']
            # Add a red rectangle for predictions
            rect = patches.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, 
                                      linewidth=2, edgecolor='red', facecolor='none')
            ax.add_patch(rect)
            ax.text(xmin, ymin - 5, f"{class_name} ({confidence:.2f})", color='red', fontsize=10, 
                    bbox=dict(facecolor='white', alpha=0.7))
    
    # Display the image with annotations
    plt.show()


# Process a specific prediction file
file_path = r"D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\PCB_DATASET\yolo_labels\01_missing_hole_10.txt"  # Update file path as needed
yolo_labels = read_yolo_labels_from_file(file_path)

# Assuming annot_df is your DataFrame with annotations (e.g., filename, width, height columns)
# annot_df = pd.read_csv('annotations.csv')  # Load the CSV containing annotations with filenames

# Convert predictions to the original annotation format
pred_annot_df = yolo_to_original_annot('01_missing_hole_10.jpg', yolo_labels, annot_df, classes)

# Display the first few rows of the converted DataFrame
print(pred_annot_df.head())

# Visualization functions
visualize_annotations('01_missing_hole_10.jpg', test_data_dir, annot_df, pred_annot_df, is_subfolder=True)

print("Inference, conversion, and visualization completed.")


# Average Confidence

Greyscale Images


In [12]:
import os

# Function to read YOLO format labels from file and extract confidence values
def read_yolo_labels_from_file(file_path):
    confidence_values = []
    with open(file_path, 'r') as file:
        for line in file:
            values = line.strip().split()
            # The confidence value is the 5th element in YOLO format (index 4)
            if len(values) == 6:  # YOLO format with confidence
                confidence_values.append(float(values[5]))
            elif len(values) == 5:  # YOLO format without confidence (assuming 1.0)
                confidence_values.append(1.0)
    return confidence_values

# Path to the directory containing prediction files
predictions_dir = r"D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\Final_Results\greyscale\predictions\labels"

# List all files in the directory
file_names = os.listdir(predictions_dir)

# Initialize variables to accumulate total confidence and number of predictions
total_confidence = 0
total_predictions = 0

# Loop through each file in the predictions directory
for file_name in file_names:
    # Check if the file is a YOLO prediction file (with .txt extension)
    if file_name.endswith('.txt'):
        file_path = os.path.join(predictions_dir, file_name)
        
        # Extract confidence values from the prediction file
        confidence_values = read_yolo_labels_from_file(file_path)
        
        # Add the confidence values to the total
        total_confidence += sum(confidence_values)
        total_predictions += len(confidence_values)

# Calculate the average confidence across all images
average_confidence = total_confidence / total_predictions if total_predictions > 0 else 0

# Print the result
print(f"Average Confidence: {average_confidence:.4f}")


Average Confidence: 0.6649


Binary Images


In [14]:
import os

# Function to read YOLO format labels from file and extract confidence values
def read_yolo_labels_from_file(file_path):
    confidence_values = []
    with open(file_path, 'r') as file:
        for line in file:
            values = line.strip().split()
            # The confidence value is the 5th element in YOLO format (index 4)
            if len(values) == 6:  # YOLO format with confidence
                confidence_values.append(float(values[5]))
            elif len(values) == 5:  # YOLO format without confidence (assuming 1.0)
                confidence_values.append(1.0)
    return confidence_values

# Path to the directory containing prediction files
predictions_dir = r"D:\BML Munjal\3rd Year\Sem 5\DIP\DIP-dataset\Final_Results\binary1\predictions\labels"

# List all files in the directory
file_names = os.listdir(predictions_dir)

# Initialize variables to accumulate total confidence and number of predictions
total_confidence = 0
total_predictions = 0

# Loop through each file in the predictions directory
for file_name in file_names:
    # Check if the file is a YOLO prediction file (with .txt extension)
    if file_name.endswith('.txt'):
        file_path = os.path.join(predictions_dir, file_name)
        
        # Extract confidence values from the prediction file
        confidence_values = read_yolo_labels_from_file(file_path)
        
        # Add the confidence values to the total
        total_confidence += sum(confidence_values)
        total_predictions += len(confidence_values)

# Calculate the average confidence across all images
average_confidence = total_confidence / total_predictions if total_predictions > 0 else 0

# Print the result
print(f"Average Confidence: {average_confidence:.4f}")


Average Confidence: 0.7564
